**Import Libraries**

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import meanderpy as mp             
from importlib import reload
reload(mp)
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
mpl.rcParams['figure.dpi'] = 500
import scipy.stats as stats          
import seaborn as sns                   
from scipy import interpolate          
import random

**Import River Data**

In [ ]:
 # retrieve the data address
os.chdir(r"/Users/braydennoh/Desktop/meanderpy/january/module2/edit/bend2")
# initial channel file
cl1=np.loadtxt('0.2.txt',delimiter=' ')  
# final channel file
cl2=np.loadtxt('10.2.txt',delimiter=' ')
# read the file array
y = cl1[:,0][::-1]
x = cl1[:,1][::-1]
y1 = cl2[:,0][::-1]
x1= cl2[:,1][::-1]
# optional - plotting the array
#plt.plot(x,y,label = 'initial')
#plt.plot(x1,y1,label = 'final')
#plt.legend(frameon=False)


**Meanderpy Manual Calibration**

In [ ]:
# model standard variables
nit = 100                       # number of iterations
W = 100.0                       # channel width (m)
D = 10.0                        # channel depth (m)
depths = D * np.ones((nit,))    # channel depths for different iterations  
pad = 0                         # padding (number of nodepoints along centerline)
deltas = 50.0                   # sampling distance along centerline           
Cfs = 0.01 * np.ones((nit,))    # chezy friction coefficient
crdist = 1.8 * W                # threshold distance at which cutoffs occur
kl = 300/(365*24*60*60.0)       # migration rate constant (m/s)
kv =  1.0e-12                   # vertical slope-dependent erosion rate constant (m/s)
dt = 0.1*(365*24*60*60.0)       # time step (s)
dens = 1000                     # density of water (kg/m3)
saved_ts = 1                    # which time steps will be saved
Sl = 0.0                        # initial slope (matters more for submarine channels than rivers)
t1 = 0                          # time step when incision starts
t2 = 0                          # time step when lateral migration starts
t3 = 0                          # time step when vertical migration starts
aggr_factor = 2e-9              # aggradation factor (m/s, about 0.18 m/year, it kicks in after t3) after t3)
sc = 1.0                        # scaling factor   
y=cl1[:,0]*10                
x=cl1[:,1]*10
z=np.zeros(len(x))
H=depths[0]
# initialize model
ch=mp.Channel(-x,-y,z,W,H)
chb=mp.ChannelBelt(channels=[ch], cutoffs=[], cl_times=[0.0], cutoff_times=[])
chb.migrate(nit,saved_ts,deltas,pad,crdist,depths,Cfs,kl,kv,dt,dens,t1,t2,t3,aggr_factor)
channel_coordinate = pd.DataFrame({"x":chb.channels[nit-1].x, "y":chb.channels[nit-1].y, "Z":chb.channels[nit-1].z})
np.savetxt('output.txt', channel_coordinate, delimiter=',')
print('You are running this simulation for:', nit*dt/(365.*24*60*60),'Years')
print('Migration rate constant =', kl*60*60.*24.*365., 'm/yr')
# make the synethetic data
synx = 0.1*chb.channels[np.int(nit-1)].x
syny = 0.1*chb.channels[np.int(nit-1)].y
np.savetxt('syn.txt', np.c_[syny,synx])
clsyn=np.loadtxt('syn.txt',delimiter=' ')
ax = pd.DataFrame({"x":clsyn[:,1], "y":clsyn[:,0]}).plot.line(x='x', y='y', label='initial actual (0 years)')
pd.DataFrame({"x":-cl1[:,1], "y":-cl1[:,0]}).plot.line(x='x', y='y', ax= ax, label='initial actual (0 years)')
pd.DataFrame({"x":-cl2[:,1], "y":-cl2[:,0]}).plot.line(x='x', y='y', ax= ax, label='final actual (10 years)')

**Interpolate Nodepoints**

In [ ]:
#interpolate initial data
t = np.linspace(0,1,np.shape(cl1[:,])[0])   # create a new variable t 
x_o = -cl1[:,1].flatten()                   # get x-axis data
y_o = -cl1[:,0].flatten()                   # get y-axis data
fx_o = interpolate.interp1d(t,x_o)          # initialize the x interpolation class
fy_o = interpolate.interp1d(t, y_o)         # initialize the y interpolation class
tnew = np.linspace(0,1,1000)                # create a finer t-variable spacing 
xnew_o = fx_o(tnew)                         # get interpolated x values
ynew_o = fy_o(tnew)                         # get interpolated y values
data_obs_ins = np.array([xnew_o,ynew_o])    # combine the x and y
data_obs_ins = np.round(data_obs_ins, 2)    # truncation
#interpolate final (or synthetic) data
t = np.linspace(0,1,np.shape(clsyn[:,])[0]) 
x_o = clsyn[:,1].flatten()                 
y_o = clsyn[:,0].flatten()                 
fx_o = interpolate.interp1d(t,x_o)        
fy_o = interpolate.interp1d(t, y_o)       
tnew = np.linspace(0,1,1000)               
xnew_o = fx_o(tnew)                       
ynew_o = fy_o(tnew)                        
data_obs = np.array([xnew_o,ynew_o])      
data_obs = np.flip(data_obs, axis=1)       
data_obs = np.round(data_obs, 1)
#interpolate simulation output data
t = np.linspace(0,1,np.shape(0.1*chb.channels[np.int(nit-1)].x)[0])
x_m = 0.1*chb.channels[np.int(nit-1)].x
y_m = 0.1*chb.channels[np.int(nit-1)].y
fx_m = interpolate.interp1d(t,x_m)
fy_m = interpolate.interp1d(t, y_m)
tnew = np.linspace(0,1,1000)
xnew_m = fx_m(tnew)
ynew_m = fy_m(tnew)
ynew_ms = ynew_m
xnew_ms = xnew_m

**Plotting Difference between Observed Final vs. Simulation**

In [ ]:
data_obs =np.flip(data_obs, axis=1) 
err1 = data_obs_ins[0,:] - data_obs[0,:]
err2 = xnew_ms - data_obs[0,:]
plt.plot(ynew_ms - data_obs[1,:])

**Setting up Meanderpy for MCMC**

In [ ]:
# model standard variables
nit = 100                   
depths = D * np.ones((nit,)) 
pad = 0                 
deltas = 50.0               
crdist = 1.8 * W              
kv =  1.0e-12             
dt = 0.1*(365*24*60*60.0)    
dens = 1000               
saved_ts = 1              
n_bends = 5              
Sl = 0.0            
t1 = 0                  
t2 = 0                  
t3 = 0      
# defining the howard knutson model for MCMC running.
# In this case, we are using 2 parameters for the model: migration rate and chezy friction factor
def hkm(parm):  
    kl =  (parm[0]*10)/(365*24*60*60.0)        # parameter 1 = migration rate
    Cfs = parm[1] * 0.001 * np.ones((nit,))    # parameter 2 = chezy friction factor
    y=cl1[:,0]*10
    x=cl1[:,1]*10
    z=np.zeros(len(x))
    H=depths[0]
    # initialize model
    try:    
        ch=mp.Channel(-x,-y,z,W,H)
        chb=mp.ChannelBelt(channels=[ch], cutoffs=[], cl_times=[0.0], cutoff_times=[])
        ch = mp.generate_initial_channel(W,D,Sl,deltas,pad,n_bends) 
        chb.migrate(nit,saved_ts,deltas,pad,crdist,depths,Cfs,kl,kv,dt,dens,t1,t2,t3,aggr_factor)
        #interpolate the output data
        if np.shape(0.1*chb.channels[np.int(nit-1)].x)[0] < 1000:
            t = np.linspace(0,1,np.shape(0.1*chb.channels[np.int(nit-1)].x)[0])
            x_m = 0.1*chb.channels[np.int(nit-1)].x
            y_m = 0.1*chb.channels[np.int(nit-1)].y
            fx_m = interpolate.interp1d(t,x_m)
            fy_m = interpolate.interp1d(t, y_m)
            tnew = np.linspace(0,1,1000)
            xnew_m = fx_m(tnew) 
            ynew_m = fy_m(tnew) 
            xnew_m[:] = xnew_m[::-1] 
            ynew_m[:] = ynew_m[::-1] 
        else:
            t = np.linspace(0,1,1000)
            xnew_m = np.zeros(1000) 
            ynew_m = np.zeros(1000) 
            ynew_m[:] = ynew_m[::-1] 
            xnew_m[:] = xnew_m[::-1] 
    except:
        t = np.linspace(0,1,1000)
        xnew_m = np.zeros(1000) 
        ynew_m = np.zeros(1000) 
        ynew_m[:] = ynew_m[::-1] 
        xnew_m[:] = xnew_m[::-1] 
    return np.array([xnew_m,ynew_m])

In [ ]:
def log_prior(par):
# define the bounds of the prior   
    if  10 < par[0] < 100 and 0 < par[1] < 30 and 0.01 < par[2] < 10:
        return -par[2]/1 # return the exponential distribution
    else:
        return -np.inf   # return the exponential distribution
# define log likelihood    
def log_lik(par,data):
    print(par)
    sim = hkm(parm=par[:2])
    x_err = sim[0,:]-data[0,:][::-1]
    y_err = sim[1,:]-data[1,:][::-1]
    plt.plot(x_err) 
    plt.plot(y_err) 
    y_fft = np.fft.fft(y_err)  # do the fast fourier transform of y-errrors
    x_fft = np.fft.fft(x_err)  # do the fast fourier transform of x-errrors
    y_amp1 = 2*abs(y_fft.real/y_fft.size)      # retrieve amplitude of cos function
    y_amp2 = 2*abs(y_fft.imag/y_fft.size)      # retrieve amplitude of sin function
    x_amp1 = 2*abs(x_fft.real/x_fft.size)      # retrieve amplitude of cos function
    x_amp2 = 2*abs(x_fft.imag/x_fft.size)      # retrieve amplitude of sin function
    amp = np.array([y_amp1,y_amp2,x_amp1,x_amp2]) # write all four amplitudes in one array 
    return np.sum(np.log(stats.norm.pdf(amp, loc=0, scale=par[2]))) # evaluate the probability density of the amp array
# define log posterior
def log_post(par, data):
    par = np.around(par, decimals = 0)
    print (par)
    if (log_prior(par)==-np.inf):
        lp = log_prior(par)
    else: 
        lp = log_lik(par,data)+ log_prior(par)
    return lp  
#test run
log_post([25,10,1],data_obs)

In [ ]:
from scipy.stats import truncnorm
import emcee
# truncated normal distribution 
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
ndim, nwalkers = 3, 20
# get parameter mean and standard devication
X1 = get_truncated_normal(mean=10, sd=15, low=0, upp=50)
X2 = get_truncated_normal(mean=5, sd=10, low=0, upp=20)
X3 = get_truncated_normal(mean=2, sd=1, low=1, upp=5)
# get MCMC nwalker and n dimension
p0 = np.ones([nwalkers, ndim])
p0[:,0] = X1.rvs(size = nwalkers)
p0[:,1] = X2.rvs(size = nwalkers)
p0[:,2] = X3.rvs(size = nwalkers)
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, args = [data_obs])

In [ ]:
# MCMC sample run
samples_runs = 20
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, args = [data_obs])
sampler.run_mcmc(p0, samples_runs) 
samp = sampler.get_chain()
# converting 3D shape to 2D shape
samp_reshaped = samp.reshape(samp.shape[0], -1) 
np.savetxt('samples_emcee.txt',samp_reshaped, delimiter=',')
sampsubset = samp[0:samples_runs,:,:]
samp2d = sampsubset.reshape(sampsubset.shape[0]*sampsubset.shape[1],3)
samp2d.shape
samp2d 

In [ ]:
# plotting migration rate parameter chain
plt.plot(sampler.chain[:,:,0].T, ':')
plt.savefig('migrations.PNG',dpi = 200)

In [ ]:
# plotting chezy friction coefficient parameter chain
plt.plot(sampler.chain[:,:,1].T, ':')
plt.savefig('migrations.PNG',dpi = 200)

In [ ]:
# plotting parameter convergence 
plt.rcParams.update({'font.size': 22})
samp_df = pd.DataFrame(samp2d)
samp_df = samp_df.rename(columns={0: 'migration rate',  1: 'chezy', 2: 'error'})   
samp_df
sns.color_palette("mako")
sns.set(style="ticks")
b = sns.pairplot(samp_df)
plt.savefig('kde%s.PNG',dpi = 300)

**Generating the Risk Map**

In [ ]:
# loading synthetic data
clin=-cl1+1000
clfi=-cl2+1000
clsyn=np.loadtxt('syn.txt',delimiter=' ')
# interpolating simulation data
t = np.linspace(0,1,np.shape(clin[:,])[0]) # create a new variable t 
x_o = clin[:,1].flatten()            # get x-axis data
y_o = clin[:,0].flatten()            # get y-axis data
fx_o = interpolate.interp1d(t,x_o)      # initialize the interpolation class
fy_o = interpolate.interp1d(t, y_o)    
tnew = np.linspace(0,1,10000)                        # create a finer t-variable spacing 
xnew_o = fx_o(tnew) + np.random.normal(0,0.2,10000) # get interpolated x values
ynew_o = fy_o(tnew) + np.random.normal(0,0.2,10000) # get interpolated y values
data_obs_ins = np.array([xnew_o,ynew_o])
data_obs_ins = np.round(data_obs_ins, 5) 
data_obs_ins = np.flip(data_obs_ins, axis=1)      
# defining variable
nit = 100                   
depths = D * np.ones((nit,)) 
pad = 0                 
deltas = 50.0               
crdist = 1.8 * W              
kv =  1.0e-12             
dt = 0.1*(365*24*60*60.0)    
dens = 1000               
saved_ts = 1              
n_bends = 5              
Sl = 0.0            
t1 = 0                  
t2 = 0                  
t3 = 0
#defining howard knutson parameter for risk mapping
def hkm_predict(parm):
    kl =  (parm[0]*10)/(365*24*60*60.0)  
    Cfs = parm[1] * 0.001 * np.ones((nit,))
    y=cl1[:,0]*10
    x=cl1[:,1]*10
    z=np.zeros(len(x))
    H=depths[0]
    try:    # use "try" to simulation doesn't stop if there is an error 
        ch=mp.Channel(-x,-y,z,W,H)
        chb=mp.ChannelBelt(channels=[ch], cutoffs=[], cl_times=[0.0], cutoff_times=[])
        ch = mp.generate_initial_channel(W,D,Sl,deltas,pad,n_bends) # initialize channel
        chb.migrate(nit,saved_ts,deltas,pad,crdist,depths,Cfs,kl,kv,dt,dens,t1,t2,t3,aggr_factor) # channel migration
        # create an if loop to discard simulations where the channel geometry 
        #is so sinuous that it has more that 10,000 points
        if np.shape(0.1*chb.channels[np.int(nit-1)].x)[0] < 10000:
            t = np.linspace(0,1,np.shape(0.1*chb.channels[np.int(nit-1)].x)[0])
            x_m = (0.1*chb.channels[np.int(nit-1)].x)+1000
            y_m = (0.1*chb.channels[np.int(nit-1)].y)+1000
            fx_m = interpolate.interp1d(t,x_m)
            fy_m = interpolate.interp1d(t, y_m)
            tnew = np.linspace(0,1,10000)
            xnew_m = fx_m(tnew)
            ynew_m = fy_m(tnew)
        # create an artificial channel which is a very poor prediction
        else:
            t = np.linspace(0,1,10000)
            xnew_m = np.zeros(10000)
            ynew_m = np.zeros(10000)
    except:
        t = np.linspace(0,1,10000)
        xnew_m = np.zeros(10000)
        ynew_m = np.zeros(10000)
    return np.array( [xnew_m,ynew_m])
# create a random noise variable
yss = np.zeros([10000,1000])
del_x = np.linspace(0,1,10000)
for i in range(0,1000):
    yss[:,i]= 1*(np.random.normal(0,1,1)*np.cos(2*np.pi*del_x)+np.random.normal(0,1,1)*np.sin(2*np.pi*del_x))
# define the pixel resolution 
sc=1/1
counts = np.zeros((int(sc*800), int(sc*800))) #create variables to store the information abt counts
prob = np.zeros((int(sc*800), int(sc*800))) #create variables to store the information abt proability
# counting mechanism starts from 0 and counts upward
counts_obs = np.zeros((int(sc*800),int(sc*800)))  #create variables to store the information abt counts
prob_obs = np.zeros((int(sc*800), int(sc*800))) #create variables to store the information abt proability
#initial river condition
ax = pd.DataFrame({"x":clin[:,1], "y":clin[:,0]}).plot.line(x='x', y='y', label='initial actual (0 years)', color='gray',lw=4)
# setting the subtitle
ax.set_xlabel("x (--)")
ax.set_ylabel("y (--)")
ax.set_title("5-year prediction for Ucayali River")
#counting mechanism loop
for i in np.random.randint(100, 400, 100): # 100 is the amount of foward running
    samplenew = samp2d[i,:2]
    sim = hkm_predict(samplenew)
    x = sim[0,:]
    y = sim[1,:]
    asdf = np.array([x,y])
    np.savetxt('samples_emcee.txt',asdf, delimiter=',')
    pd.DataFrame({"x":sim[0,:], "y":sim[1,:]}).plot.line(x='x', y='y', ax= ax, color='red',label='_Hidden')
    for j in range(0,int(sc*800)):
        # get the x-values of the river corresponding to the y-value of a certain
        # horizontal line will be 800 by 800 grid cell
        idx = np.argwhere(np.diff(np.sign(sim[1,:] - np.arange(200,1000,1/sc)[j]))).flatten() 
        idx_obs = np.argwhere(np.diff(np.sign(data_obs_ins[1,:] - np.arange(200,1000,1/sc)[j]))).flatten() 
        #ax1.plot(x[idx], y[idx], 'ro')
        intersection = sim[0,idx]
        intersection_obs = data_obs_ins[0,idx_obs]
        #ax1.set_xticks(np.arange(-20, 21, 2))
        #ax1.set_yticks(np.arange(-20, 21, 2))   
        # vertical line will be 800 by 800 grid cell
        for k in range(0,int(sc*800)):
            # count the intersection points on the left side of the grid cell
            counts[k,j]= np.shape(intersection[intersection<=np.arange(200,1000,1/sc)[k]])[0]
            counts_obs[k,j]= np.shape(intersection_obs[intersection_obs<=np.arange(200,1000,1/sc)[k]])[0]
    counts = np.absolute(counts-counts_obs)
    #counts = (counts-counts_obs)
    counts[counts%2 == 1] = 1 # erosion happened i.e. = 1 if count number even
    counts[counts%2 == 0] = 0 # no erosion i.e. = 0 if count number off
    #counts[counts != 0] = 1 # erosion happened i.e. = 1 if count number even
    prob = prob + counts  # add up all the "monte carlo" runs that pass the grid cell
pd.DataFrame({"x":sim[0,:], "y":sim[1,:]}).plot.line(x='x', y='y', ax= ax, color='red',label="probabilistic prediction (5 years)")
pd.DataFrame({"x":clfi[:,1], "y":clfi[:,0]}).plot.line(x='x', y='y', ax= ax, label='final actual (5 years)',color='black',lw=4)
plt.savefig('prob_prediction.jpg')
prob = prob/100 # divide by the forward run time, which is 100 in this case
y, x = np.meshgrid(np.arange(0,800,1/sc), np.arange(0,800,1/sc)) #create a 800 by 800 meshgrid
z = prob #define probablity z
z_min, z_max = 0, np.abs(z).max()
fig, ax = plt.subplots(figsize=(5,4))

c = ax.pcolormesh(x, y, z, cmap='jet', vmin=z_min, vmax=z_max)
fig.colorbar(c, ax=ax)
fig.legend('',frameon=False)
plt.xlim([400, 750])
plt.ylim([250, 600])

mpl.rcParams.update({'text.color' : "white",
                     'axes.labelcolor' : "blue"})
d = z.ravel()
pd.DataFrame({"x":-cl1[:,1]+800, "y":-cl1[:,0]+800}).plot.line(x='x', y='y', ax= ax,  color='lightblue',lw = 2,label='initial actual (0 years)')
pd.DataFrame({"x":clsyn[:,1]+800, "y":clsyn[:,0]+800}).plot.line(x='x', y='y',   color='red',lw =2 , ax= ax,label='final actual (10 years)')
plt.legend(frameon=False,labelcolor='white')
p1 = ((0 <= d) & (d <= 0.25)).sum()
p2 = ((0.25 <= d) & (d <= 0.75)).sum()
p3 = ((0.75 <= d) & (d <= 1)).sum()
print (p1)
print (p2)
print (p3)
fig.savefig('risk%s.PNG'% i,dpi = 200)